In [38]:
!pip install seqeval

In [39]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from gensim.models import Word2Vec
from seqeval.metrics import classification_report, f1_score
from tqdm import tqdm
import os

### `CNN Classification`

In [40]:
SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 100  # Must match your TP1 Model dimensions
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
W2V_PATH = "w2v_med_cbow.model"

TRAIN_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtrain_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtrain_layer1_ID.csv",
]
VALID_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEdev_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAdev_layer1_ID.csv",
]
TEST_FILES = [
    "TP_ISD2020/QUAERO_FrenchMed/MEDLINE/MEDLINEtest_layer1_ID.csv",
    "TP_ISD2020/QUAERO_FrenchMed/EMEA/EMEAtest_layer1_ID.csv",
]

In [42]:
# --- 2. ROBUST DATA LOADER ---
def load_data_from_csv(file_paths):
    all_sentences = []
    all_tags = []

    for fpath in file_paths:
        if not os.path.exists(fpath):
            print(f"❌ File not found: {fpath}")
            continue
        print(f"Loading {os.path.basename(fpath)}...", end=" ")

        try:
            # key: skip_blank_lines=False preserves sentence separators
            df = pd.read_csv(
                fpath,
                sep=None,
                engine="python",
                keep_default_na=False,
                skip_blank_lines=False,
            )
        except:
            print("Read Failed.")
            continue

        # Detect columns
        if "Mot" in df.columns and "Tag" in df.columns:
            words, tags = df["Mot"].astype(str).values, df["Tag"].astype(str).values
        else:
            words, tags = (
                df.iloc[:, 0].astype(str).values,
                df.iloc[:, -1].astype(str).values,
            )

        curr_s, curr_t = [], []
        file_s, file_t = [], []

        for w, t in zip(words, tags):
            if not w.strip():  # Empty line = Sentence Break
                if curr_s:
                    file_s.append(curr_s)
                    file_t.append(curr_t)
                    curr_s, curr_t = [], []
            else:
                curr_s.append(w)
                curr_t.append(t)
        if curr_s:
            file_s.append(curr_s)
            file_t.append(curr_t)

        # Fallback for giant files (Chunking)
        if len(file_s) < 10 and len(words) > 500:
            flat_w = [w for s in file_s for w in s]
            flat_t = [t for s in file_t for t in s]
            file_s = [
                flat_w[i : i + SEQUENCE_LENGTH]
                for i in range(0, len(flat_w), SEQUENCE_LENGTH)
            ]
            file_t = [
                flat_t[i : i + SEQUENCE_LENGTH]
                for i in range(0, len(flat_t), SEQUENCE_LENGTH)
            ]

        print(f"-> {len(file_s)} sentences.")
        all_sentences.extend(file_s)
        all_tags.extend(file_t)

    return all_sentences, all_tags


print("--- LOADING DATA ---")
train_sents, train_tags = load_data_from_csv(TRAIN_FILES)
valid_sents, valid_tags = load_data_from_csv(VALID_FILES)
test_sents, test_tags = load_data_from_csv(TEST_FILES)

--- LOADING DATA ---
Loading MEDLINEtrain_layer1_ID.csv... -> 91 sentences.
Loading EMEAtrain_layer1_ID.csv... -> 120 sentences.
Loading MEDLINEdev_layer1_ID.csv... -> 90 sentences.
Loading EMEAdev_layer1_ID.csv... -> 106 sentences.
Loading MEDLINEtest_layer1_ID.csv... -> 94 sentences.
Loading EMEAtest_layer1_ID.csv... -> 97 sentences.


In [43]:
# --- 3. BUILD VOCAB & INITIALIZE EMBEDDINGS (TP1) ---
print("\n--- INITIALIZING EMBEDDINGS ---")

# A. Build Vocabulary from Training Data
word_counts = {}
for sent in train_sents:
    for word in sent:
        word_counts[word] = word_counts.get(word, 0) + 1

vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word2idx = {w: i + 2 for i, w in enumerate(vocab)}  # Start at 2
word2idx["<PAD>"] = 0
word2idx["<UNK>"] = 1
idx2word = {v: k for k, v in word2idx.items()}

# B. Load Word2Vec and Create Weight Matrix
w2v_model = Word2Vec.load(W2V_PATH)
vocab_size = len(word2idx)
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
hits = 0

for word, idx in word2idx.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
        hits += 1
    elif word.lower() in w2v_model.wv:  # Case Insensitive Fallback
        embedding_matrix[idx] = w2v_model.wv[word.lower()]
        hits += 1
    else:
        # Initialize OOV words with random noise
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM,))

print(f"Vocab Size: {vocab_size}")
print(f"Pre-trained Weights Found: {hits}/{vocab_size} ({hits/vocab_size:.1%})")

# Convert to FloatTensor
embedding_weights = torch.tensor(embedding_matrix, dtype=torch.float32).to(DEVICE)


--- INITIALIZING EMBEDDINGS ---
Vocab Size: 5775
Pre-trained Weights Found: 5621/5775 (97.3%)


In [44]:
# --- 4. DATA PREPARATION (ENCODING) ---
def encode_sequences(seqs, mapping, default_val, max_len=128):
    encoded = []
    for s in seqs:
        seq = [mapping.get(item, default_val) for item in s]
        if len(seq) < max_len:
            seq += [0] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        encoded.append(seq)
    return torch.tensor(encoded, dtype=torch.long)


# Encode Words (Input)
X_train = encode_sequences(train_sents, word2idx, 1)  # 1 = UNK
X_valid = encode_sequences(valid_sents, word2idx, 1)
X_test = encode_sequences(test_sents, word2idx, 1)

# Encode Tags (Target)
tag_set = set(t for s in train_tags + valid_tags + test_tags for t in s)
tag2idx = {t: i + 1 for i, t in enumerate(sorted(list(tag_set)))}
tag2idx["<PAD>"] = 0
idx2tag = {v: k for k, v in tag2idx.items()}
print(f"Tags: {tag2idx}")

y_train = encode_sequences(train_tags, tag2idx, 0)  # 0 = PAD (ignored in loss)
y_valid = encode_sequences(valid_tags, tag2idx, 0)
y_test = encode_sequences(test_tags, tag2idx, 0)

# Loaders
train_loader = DataLoader(
    TensorDataset(X_train, y_train), shuffle=True, batch_size=BATCH_SIZE
)
valid_loader = DataLoader(
    TensorDataset(X_valid, y_valid), shuffle=False, batch_size=BATCH_SIZE
)
test_loader = DataLoader(
    TensorDataset(X_test, y_test), shuffle=False, batch_size=BATCH_SIZE
)

Tags: {'B-ANAT': 1, 'B-CHEM': 2, 'B-DEVI': 3, 'B-DISO': 4, 'B-GEOG': 5, 'B-LIVB': 6, 'B-OBJC': 7, 'B-PHEN': 8, 'B-PHYS': 9, 'B-PROC': 10, 'I-ANAT': 11, 'I-CHEM': 12, 'I-DEVI': 13, 'I-DISO': 14, 'I-GEOG': 15, 'I-LIVB': 16, 'I-OBJC': 17, 'I-PHEN': 18, 'I-PHYS': 19, 'I-PROC': 20, 'O': 21, '<PAD>': 0}


In [45]:
# --- 5. CNN MODEL ---
class CNN_NER(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, pretrained_weights):
        super(CNN_NER, self).__init__()

        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # INITIALIZATION: Overwrite with TP1 weights
        self.embedding.weight.data.copy_(pretrained_weights)

        # CNN Layers (Conv1d preserves sequence length with padding=1)
        self.conv1 = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.embedding(x)  # [Batch, Seq, Dim]
        x = x.permute(0, 2, 1)  # [Batch, Dim, Seq] (Required for Conv1d)

        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = F.relu(self.conv2(x))
        x = self.dropout(x)

        x = x.permute(0, 2, 1)  # [Batch, Seq, 256]
        return self.fc(x)


model = CNN_NER(vocab_size, EMBEDDING_DIM, len(tag2idx), embedding_weights).to(DEVICE)

In [46]:
# --- 6. TRAINING ---
# Weighted Loss for "O" Class Imbalance
weights = torch.ones(len(tag2idx)).to(DEVICE)
if "O" in tag2idx:
    weights[tag2idx["O"]] = 0.5
criterion = nn.CrossEntropyLoss(weight=weights, ignore_index=0)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

print(f"\n--- STARTING TRAINING ON {DEVICE} ---")
best_f1 = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out.view(-1, len(tag2idx)), y.view(-1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    all_true, all_pred = [], []
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            preds = torch.argmax(out, dim=2).cpu().numpy()
            labels = y.cpu().numpy()

            for i in range(len(x)):
                p_s, t_s = [], []
                for j in range(SEQUENCE_LENGTH):
                    if labels[i][j] == 0:
                        break  # Stop at PAD
                    p_s.append(idx2tag[preds[i][j]])
                    t_s.append(idx2tag[labels[i][j]])
                all_pred.append(p_s)
                all_true.append(t_s)

    val_f1 = f1_score(all_true, all_pred)
    print(f"Loss: {train_loss/len(train_loader):.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_cnn_ner.pt")


--- STARTING TRAINING ON cpu ---


Epoch 1: 100%|██████████| 7/7 [00:01<00:00,  3.93it/s]


Loss: 2.5214 | Val F1: 0.0000


Epoch 2: 100%|██████████| 7/7 [00:01<00:00,  4.80it/s]


Loss: 1.8575 | Val F1: 0.0000


...

Loss: 0.0497 | Val F1: 0.3141


Epoch 99: 100%|██████████| 7/7 [00:01<00:00,  5.25it/s]


Loss: 0.0518 | Val F1: 0.3147


Epoch 100: 100%|██████████| 7/7 [00:01<00:00,  4.33it/s]


Loss: 0.0500 | Val F1: 0.3168


In [47]:
# --- 7. FINAL TEST EVALUATION ---
print("\n--- TEST RESULTS ---")
model.load_state_dict(torch.load("best_cnn_ner.pt"))
model.eval()
test_true, test_pred = [], []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)
        preds = torch.argmax(out, dim=2).cpu().numpy()
        labels = y.numpy()
        for i in range(len(x)):
            p_s, t_s = [], []
            for j in range(SEQUENCE_LENGTH):
                if labels[i][j] == 0:
                    break
                p_s.append(idx2tag[preds[i][j]])
                t_s.append(idx2tag[labels[i][j]])
            test_pred.append(p_s)
            test_true.append(t_s)

print(classification_report(test_true, test_pred))


--- TEST RESULTS ---


              precision    recall  f1-score   support

        ANAT       0.57      0.01      0.02       364
        CHEM       0.37      0.29      0.33      1037
        DEVI       0.00      0.00      0.00       107
        DISO       0.27      0.26      0.26       977
        GEOG       0.00      0.00      0.00        63
        LIVB       0.77      0.49      0.60       498
        OBJC       0.00      0.00      0.00        81
        PHEN       0.00      0.00      0.00        70
        PHYS       0.00      0.00      0.00       190
        PROC       0.30      0.39      0.34       761

   micro avg       0.36      0.27      0.31      4148
   macro avg       0.23      0.14      0.16      4148
weighted avg       0.35      0.27      0.28      4148



/Users/idrissamahamoudoudicko/anaconda3/envs/HandonNLP/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LSTM